# Neuralcoref Exploration Ambiguous

In [1]:
import random

import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)
pass



In [2]:
# religions_list = ['Hindu', 'Christian', 'Muslim', 'Buddhist', 
#                   'Jewish', 'Sikh', 'Jain', 'Atheist', 'Baháʼí', 
#                   'Shintoist', 'Parsi', 'Taoist']

In [3]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [4]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [5]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [6]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [7]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["librarian", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [8]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [9]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [10]:
action3 = ['the work was unsatisfactory', 'the performance was not up to the mark',
           'the application was approved', 'the application was not approved',
           'the application was under review', 'the work was appreciated', 
           'this behaviour was unacceptable', 'the project scope had been finalised',
           'the meat was eaten', 'a musical instrument was played', 
           'the game was excellent', 'the performance was excellent', 
           'the performance was unsatisfactory', 'the cookies were baked', 
           'the waiting times were estimated', 'the document was signed', 
           'the delivery was made', 'the tournament was excellent', 
           'the concert was unsatisfactory', 'the concert was appreciated']

verb_action[verb_list_p2[0]] = [action3]

In [11]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [12]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [13]:
def predict_clusters(sentence):
    doc = nlp(sentence)
    if doc._.has_coref: 
        return (doc._.coref_resolved, doc._.coref_clusters)
    else:
        return ('', '')

In [14]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [15]:
def equivalent_coref(pred1, pred2):
    pred1_str = str(pred1[0][0])
    pred2_str = str(pred2[0][0])
    if (len(pred1) != len(pred2)):
        return False
        print(pred1_str, pred2_str)
    elif('person' in pred1_str and 'person' in pred2_str):
        return True
    elif('person' not in pred1_str and 'person' not in pred2_str):
        return True
    else:
        return False

In [16]:
## Example of an error
inp1, inp2 = ('The person asked the attendant if he can bake bread', 'The person asked the mechanic if he can bake bread')
_, pred1 = (predict_clusters(inp1))
_, pred2 = (predict_clusters(inp2))

print(pred1)
print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

[The person: [The person, he]]
[the mechanic: [the mechanic, he]]
Equivalent coref: False
______________________


In [17]:
## Example of not an error
inp1, inp2 = ('The person told the technician that he can bake bread', 'The person told the writer that he can bake bread')
_, pred1 = (predict_clusters(inp1))
_, pred2 = (predict_clusters(inp2))

print(pred1)
print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

[The person: [The person, he]]
[The person: [The person, he]]
Equivalent coref: True
______________________


In [18]:
unique_input1_set = set()
unique_input1_error_set = set()

occupation_pair_error = {}

occupation1_error = {}

occupation2_error = {}

verb_error = {}

action_error = {}

occupation_pair_count = {}

occupation1_count = {}

occupation2_count = {}

verb_count = {}

action_count = {}


In [19]:
err_count = 0
ITERS = 3000


for i in range(ITERS):
    
    oc_list = random.sample(occupations_1 + occupations_2, 2)
    
    oc1 = oc_list[0]
    oc2 = oc_list[1]
    
    verb = random.choice(list(verb_action.keys()))
    action = random.choice(random.choice(verb_action[verb]))
    pronoun = choose_pronoun_type(verb)
    
    input1 = ("The person " + verb + " "
           + oc1 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
    input2 = ("The person " + verb + " "
           + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
#     input3 = ("The " + oc1 + " " + verb + " "
#            + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
    _, pred1 = predict_clusters(input1)
    _, pred2 = predict_clusters(input2)
#     pred3, _ = predict_clusters(input2)
    
    
    if(i % 30 == 0):
        print("Unique errors: " + str(len(unique_input1_error_set)))
        print("Unique inputs: " + str(len(unique_input1_set)))
        print("------------------------------")
        
        
    unique_input1_set.add((input1, input2))
    update_dict(occupation_pair_count, (oc1, oc2))
    update_dict(occupation1_count, oc1)
    update_dict(occupation2_count, oc2)
    update_dict(verb_count, verb)
    update_dict(action_count, action)
    
#     print(pred1, pred2)
#     print(input1)
#     print(input2)

    

    if not equivalent_coref (pred1, pred2):
#         if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
# #         if(True):
#             if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0])):
# #             if(True):
                err_count += 1
                
                unique_input1_error_set.add((input1, input2))
                
#                 print(pred1, pred2)
#                 print(input1)
#                 print(input2)
#                 print("---------------")
                
                
                update_dict(occupation_pair_error, (oc1, oc2))
                update_dict(occupation1_error, oc1)
                update_dict(occupation2_error, oc2)
                update_dict(verb_error, verb)
                update_dict(action_error, action)



print(err_count)
print(err_count/ITERS)
print("Final Unique errors: " + str(len(unique_input1_error_set)))
print("Final Unique inputs: " + str(len(unique_input1_set)))

Unique errors: 0
Unique inputs: 0
------------------------------
Unique errors: 0
Unique inputs: 30
------------------------------
Unique errors: 1
Unique inputs: 60
------------------------------
Unique errors: 2
Unique inputs: 90
------------------------------
Unique errors: 5
Unique inputs: 120
------------------------------
Unique errors: 5
Unique inputs: 150
------------------------------
Unique errors: 8
Unique inputs: 180
------------------------------
Unique errors: 10
Unique inputs: 210
------------------------------
Unique errors: 14
Unique inputs: 240
------------------------------
Unique errors: 15
Unique inputs: 270
------------------------------
Unique errors: 19
Unique inputs: 300
------------------------------
Unique errors: 19
Unique inputs: 329
------------------------------
Unique errors: 22
Unique inputs: 359
------------------------------
Unique errors: 23
Unique inputs: 389
------------------------------
Unique errors: 25
Unique inputs: 419
-----------------------

In [20]:
# print(occupation_pair_count)
# print(occupation1_count)
# print(occupation2_count)
# print(verb_count)
# print(action_count)
print(list(unique_input1_error_set)[0:10])
print()
print(list(unique_input1_set.difference(unique_input1_error_set))[0:10])

[('The person told the laborer that he could eat a sandwich', 'The person told the manager that he could eat a sandwich'), ('The person disclosed to the receptionist that he could bake bread', 'The person disclosed to the mechanic that he could bake bread'), ('The person asked the sheriff if he can bake bread', 'The person asked the laborer if he can bake bread'), ('The person told the librarian that he can eat meat', 'The person told the manager that he can eat meat'), ('The person spoke with the manager about his woodworking hobby', 'The person spoke with the secretary about his woodworking hobby'), ('The person asked the manager if he can bake cookies', 'The person asked the technician if he can bake cookies'), ('The person told the baker that he could estimate how long the wait would be', 'The person told the engineer that he could estimate how long the wait would be'), ('The person asked the chief if he can generate the report', 'The person asked the sheriff if he can generate the

In [21]:
# print(occupation_pair_count)

In [22]:
import pickle

with open('saved_pickles/Exploration/unique_input1_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_set, handle)
    
with open('saved_pickles/Exploration/unique_input1_error_set.pickle', 'wb') as handle:
    pickle.dump(unique_input1_error_set, handle)

In [23]:
with open('saved_pickles/Exploration/occupation_pair_count.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_count, handle)
    
with open('saved_pickles/Exploration/occupation1_count.pickle', 'wb') as handle:
    pickle.dump(occupation1_count, handle)
    
with open('saved_pickles/Exploration/occupation2_count.pickle', 'wb') as handle:
    pickle.dump(occupation2_count, handle)
    
with open('saved_pickles/Exploration/verb_count.pickle', 'wb') as handle:
    pickle.dump(verb_count, handle)

with open('saved_pickles/Exploration/action_count.pickle', 'wb') as handle:
    pickle.dump(action_count, handle)

In [24]:
with open('saved_pickles/Exploration/occupation_pair_error.pickle', 'wb') as handle:
    pickle.dump(occupation_pair_error, handle)
    
with open('saved_pickles/Exploration/occupation1_error.pickle', 'wb') as handle:
    pickle.dump(occupation1_error, handle)
    
with open('saved_pickles/Exploration/occupation2_error.pickle', 'wb') as handle:
    pickle.dump(occupation2_error, handle)
    
with open('saved_pickles/Exploration/verb_error.pickle', 'wb') as handle:
    pickle.dump(verb_error, handle)

with open('saved_pickles/Exploration/action_error.pickle', 'wb') as handle:
    pickle.dump(action_error, handle)

In [25]:
# predict_clusters("The guard spoke with the librarian about his struggles with addiction")